In [18]:
!pip install gspread-formatting --quiet
print("gspreaed formatting successfully installed!")

gspreaed formatting successfully installed!


In [77]:
from google.cloud import bigquery
from google.oauth2 import service_account
import gspread
from gspread_dataframe import set_with_dataframe
from gspread_formatting import set_column_width
import pandas as pd
import traceback

# ========= CONFIG ==========
SERVICE_ACCOUNT_FILE = '/content/drive/MyDrive/Colab Notebooks/MerchNet/MerchNet Insights.json'
PROJECT_ID = 'red-parity-456515-t4'
SPREADSHEET_NAME = 'MerchNet'
SCOPES = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/bigquery'
]
# ===========================

def get_column_letter(col_idx):
    letters = ''
    while col_idx >= 0:
        letters = chr(col_idx % 26 + 65) + letters
        col_idx = col_idx // 26 - 1
    return letters

def update_ethereum_sheet():
    try:
        credentials = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE,
            scopes=SCOPES
        )
        bq_client = bigquery.Client(credentials=credentials, project=PROJECT_ID)
        gs_client = gspread.authorize(credentials)
        spreadsheet = gs_client.open(SPREADSHEET_NAME)

        try:
            worksheet = spreadsheet.worksheet("Ethereum")
        except gspread.exceptions.WorksheetNotFound:
            worksheet = spreadsheet.add_worksheet(title="Ethereum", rows="100", cols="20")

        # Query latest PYUSD transactions
        query = """
        SELECT
          block_timestamp AS `Block Timestamp`,
          LOWER(from_address) AS `Sender's Address`,
          LOWER(to_address) AS `Receiver's Address`,
          CAST(value AS FLOAT64) / 1e6 AS Value,
          token_address AS `Contract Address`,
          transaction_hash AS `Transaction Hash`,
          block_number AS `Block`
        FROM `bigquery-public-data.crypto_ethereum.token_transfers`
        WHERE token_address = '0x6c3ea9036406852006290770bedfcaba0e23a0e8'
        ORDER BY block_timestamp DESC
        LIMIT 100
        """
        df = bq_client.query(query).to_dataframe()

        df["Etherscan Link"] = df["Transaction Hash"].map(
            lambda tx: f'=HYPERLINK("https://etherscan.io/tx/{tx}", "View Transaction")'
        )
        df["Amount in $PYUSD"] = df["Value"].map(lambda x: "{:,.2f}".format(x))
        df.drop(columns=["Value"], inplace=True)

        columns = [
            "Block Timestamp", "Sender's Address", "Receiver's Address",
            "Amount in $PYUSD", "Contract Address", "Transaction Hash", "Block", "Etherscan Link"
        ]
        df = df[columns]

        df_padded = df.copy()
        for col in df.columns:
            if col != "Etherscan Link":
                df_padded[col] = df[col].map(lambda x: f"          {x}" if pd.notnull(x) else "")

        worksheet.clear()
        worksheet.update(values=[['📊 MerchNet – Live $PYUSD Ethereum Transactions']], range_name='A1')

        num_cols = len(df.columns)
        row_count = len(df) + 2  # Header rows + data
        col_count = num_cols

        # Unmerge and re-merge top row
        spreadsheet.batch_update({
            "requests": [
                {
                    "unmergeCells": {
                        "range": {
                            "sheetId": worksheet.id,
                            "startRowIndex": 0,
                            "endRowIndex": 1,
                            "startColumnIndex": 0,
                            "endColumnIndex": num_cols
                        }
                    }
                },
                {
                    "mergeCells": {
                        "range": {
                            "sheetId": worksheet.id,
                            "startRowIndex": 0,
                            "endRowIndex": 1,
                            "startColumnIndex": 0,
                            "endColumnIndex": num_cols
                        },
                        "mergeType": "MERGE_ALL"
                    }
                },
                {
                    "repeatCell": {
                        "range": {
                            "sheetId": worksheet.id,
                            "startRowIndex": 0,
                            "endRowIndex": 1,
                            "startColumnIndex": 0,
                            "endColumnIndex": num_cols
                        },
                        "cell": {
                            "userEnteredFormat": {
                                "horizontalAlignment": "CENTER",
                                "textFormat": {
                                    "bold": True,
                                    "fontSize": 18
                                }
                            }
                        },
                        "fields": "userEnteredFormat(horizontalAlignment,textFormat)"
                    }
                }
            ]
        })

        set_with_dataframe(worksheet, df_padded, row=2)

        worksheet.format("A2:H2", {
            'textFormat': {'bold': True},
            'horizontalAlignment': 'CENTER'
        })

        for i, col in enumerate(df.columns):
            try:
                max_len = max(len(str(col)), df_padded[col].astype(str).apply(len).max()) + 2
                col_letter = get_column_letter(i)
                set_column_width(worksheet, f'{col_letter}:{col_letter}', max(100, min(max_len * 7, 400)))
            except Exception as e:
                print(f"⚠️ Skipping column width for {col}: {e}")

        # 💡 Remove old banding and apply new alternating colors
        try:
            spreadsheet.batch_update({
                "requests": [
                    {
                        "deleteBanding": {
                            "bandedRangeId": 1  # This may vary if you’ve added multiple
                        }
                    }
                ]
            })
        except:
            pass  # No existing banding to delete

        spreadsheet.batch_update({
            "requests": [
                {
                    "addBanding": {
                        "bandedRange": {
                            "range": {
                                "sheetId": worksheet.id,
                                "startRowIndex": 2,
                                "endRowIndex": row_count,
                                "startColumnIndex": 0,
                                "endColumnIndex": col_count
                            },
                            "rowProperties": {
                                "headerColor": {
                                    "red": 0.9,
                                    "green": 0.9,
                                    "blue": 0.9
                                },
                                "firstBandColor": {
                                    "red": 1.0,
                                    "green": 1.0,
                                    "blue": 1.0
                                },
                                "secondBandColor": {
                                    "red": 0.96,
                                    "green": 0.96,
                                    "blue": 0.96
                                }
                            }
                        }
                    }
                }
            ]
        })

        print("✅ Ethereum sheet updated successfully with alternating colors!")

    except Exception as e:
        print("❌ Error during sheet update:")
        traceback.print_exc()

# Run it
update_ethereum_sheet()


❌ Error during sheet update:


Traceback (most recent call last):
  File "<ipython-input-77-cb1111fb227c>", line 162, in update_ethereum_sheet
    spreadsheet.batch_update({
  File "/usr/local/lib/python3.11/dist-packages/gspread/spreadsheet.py", line 101, in batch_update
    return self.client.batch_update(self.id, body)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gspread/http_client.py", line 139, in batch_update
    r = self.request("post", SPREADSHEET_BATCH_UPDATE_URL % id, json=body)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gspread/http_client.py", line 128, in request
    raise APIError(response)
gspread.exceptions.APIError: APIError: [400]: Invalid requests[0].addBanding: You cannot add alternating background colors to a range that already has alternating background colors.
